In [47]:
import os
import csv
from matplotlib import pyplot as plt

samples=[]
with open('./data/driving_log.csv') as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        samples.append(line)
samples.pop(0)

['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']

In [31]:
from sklearn.model_selection import train_test_split
train_samples,validation_samples=train_test_split(samples,test_size=0.2)

In [38]:
import cv2
import numpy as np
import sklearn
from scipy.misc import imread

In [39]:
def generator(samples,batch_size=32):
    num_samples=len(samples)
    correction=0.2
    while 1:
#         shuffle(samples)
        for offset in range(0,num_samples,batch_size):
            batch_samples=samples[offset:offset+batch_size]
            
            images=[]
            angles=[]
            for batch_sample in batch_samples:
                name="./data/IMG/"+batch_sample[0].split('/')[-1]
                left_name="./data/IMG/"+batch_sample[1].split('/')[-1]
                right_name="./data/IMG/"+batch_sample[2].split('/')[-1]
                left_image=imread(left_name)
                right_image=imread(right_name)
                center_image=imread(name)
#                 left_image=cv2.cvtColor(left_image,cv2.COLOR_BGR2RGB)
#                 right_image=cv2.cvtColor(right_image,cv2.COLOR_BGR2RGB)
#                 center_image=cv2.cvtColor(center_image,cv2.COLOR_BGR2RGB)
                center_angle=float(batch_sample[3])
               
                left_angle=center_angle+correction
                right_angle=center_angle-correction
                
                image_flipped = cv2.flip(center_image, 1)
                measurement_flipped = -center_angle
                
                image_flipped_left = cv2.flip(left_image, 1)
                measurement_flipped_left = -left_angle
                
                image_flipped_right = cv2.flip(right_image, 1)
                measurement_flipped_right = -right_angle
                
                images.append(center_image)
                angles.append(center_angle)
                
                images.append(image_flipped)
                angles.append(measurement_flipped)
                
                images.append(image_flipped_left)
                angles.append(measurement_flipped_left)
                
                images.append(image_flipped_right)
                angles.append(measurement_flipped_right)                
                
                images.append(left_image)
                angles.append(left_angle)
                
                images.append(right_image)
                angles.append(right_angle)
            
            X_train=np.array(images)
            y_train=np.array(angles)
            yield sklearn.utils.shuffle(X_train,y_train)

In [40]:
train_generator=generator(train_samples,batch_size=32)
validation_generator=generator(validation_samples,batch_size=32)

In [41]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,Cropping2D,BatchNormalization,Activation,Dropout,MaxPooling2D,Lambda
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [42]:
# ACTIVATION='elu'
# model = Sequential()
# model.add(Lambda(lambda x: (x / 127.5) - 1., input_shape = (160, 320, 3)))
# model.add(Cropping2D(cropping=((74,24), (0,0))))
# model.add(Conv2D(24,5,5,subsample=(2,2),activation=ACTIVATION))
# model.add(Conv2D(36,5,5,subsample=(2,2),activation=ACTIVATION))
# model.add(Conv2D(48,5,5,subsample=(2,2),activation=ACTIVATION))
# model.add(Conv2D(64,3,3,activation=ACTIVATION))
# model.add(Conv2D(64,3,3,activation=ACTIVATION))
# model.add(Flatten())
# model.add(Dropout(.5))
# model.add(Dense(1164))
# model.add(Activation(ACTIVATION))
# model.add(Dense(100))
# model.add(Activation(ACTIVATION))
# model.add(Dense(50))
# model.add(Activation(ACTIVATION))
# model.add(Dense(10))
# model.add(Activation(ACTIVATION))
# model.add(Dense(1))
# model.summary()
model=Sequential()
model.add(Lambda(lambda x: x/127.5-1.0, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((74,24),(0,0))))
model.add(Conv2D(24, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(36, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(48, 5, 5, activation='elu', subsample=(2, 2)))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Conv2D(64, 3, 3, activation='elu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='elu'))
model.add(Dense(50, activation='elu'))
model.add(Dense(10, activation='elu'))
model.add(Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_6 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_6 (Cropping2D)    (None, 62, 320, 3)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 29, 158, 24)       1824      
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 13, 77, 36)        21636     
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 5, 37, 48)         43248     
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 3, 35, 64)         27712     
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 1, 33, 64)         36928     
__________

c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="elu", strides=(2, 2))`
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="elu", strides=(2, 2))`
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (5, 5), activation="elu", strides=(2, 2))`
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="elu")`


In [43]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=2)
checkpointer = ModelCheckpoint(filepath='model-{val_loss:.5f}.h5', verbose=1, save_best_only=True)
model.compile(loss='mse', optimizer='adam')
fit_loss = model.fit_generator(train_generator, steps_per_epoch=len(train_samples),
                    validation_data=validation_generator,
                               validation_steps=len(validation_samples),
                               nb_epoch=10,callbacks=[early_stopping,checkpointer],verbose=1)


c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=6429, validation_data=<generator..., validation_steps=1608, callbacks=[<keras.ca..., verbose=1, epochs=10)`
  import sys


Epoch 1/10
 160/6429 [..............................] - ETA: 3:04:10 - loss: 0.0518

Exception in thread Thread-10:
Traceback (most recent call last):
  File "c:\programdata\anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "c:\programdata\anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "c:\programdata\anaconda3\lib\site-packages\keras\utils\data_utils.py", line 579, in data_generator_task
    generator_output = next(self._generator)
  File "<ipython-input-39-896ca3f5a5f3>", line 15, in generator
    left_image=imread(left_name)
  File "c:\programdata\anaconda3\lib\site-packages\scipy\misc\pilutil.py", line 156, in imread
    im = Image.open(name)
  File "c:\programdata\anaconda3\lib\site-packages\PIL\Image.py", line 2580, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: './data/IMG/left'



 161/6429 [..............................] - ETA: 3:04:03 - loss: 0.0516

StopIteration: 

In [ ]:
plt.plot(fit_loss.history['loss'])
plt.plot(fit_loss.history['val_loss'])
plt.title('Mean Squared Error Loss')
plt.ylabel('mean squared error')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [ ]:
model.save('model.h5')